**Wikipedia spanish language model** 

In [ ]:
!pip install -q gwpy

In [ ]:
import zipfile
from google.colab import drive
from fastai.text import *
import os

In [ ]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!rm -rf wikipedia_articles
!rm -rf w_train

In [ ]:
#%%capture to not show cell output-- too long
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

In [ ]:
!mkdir w_train
!mkdir w_test

mkdir: cannot create directory ‘w_test’: File exists


In [ ]:
import re

path="/content/wikipedia_articles/"

listt = os.listdir(path)

#for filename in listt:
#  os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(" ","_")))

for filename in listt:
  os.rename(os.path.join(path, filename), os.path.join(path, re.sub('[^A-Za-z0-9]+(?![^.]*$)', '',filename)))

In [ ]:
!ls "/content/wikipedia_articles"

In [ ]:
number_files = len(listt)
train_number=round(number_files*0.85)
test_number=number_files-train_number

print("training_set_total={} ".format(train_number))
print("test_set_total={} ".format(test_number))

training_set_total=103748 
test_set_total=18308 


In [49]:
def split_sets(total_count,destiny):
  
  count=0
  for file in os.listdir(path):
      if count<total_count:
          dir="{}{}".format(path,file)
          !mv $dir $destiny  
          count+=1
          print(count)

In [ ]:
split_sets(train_number,"/content/w_train")


In [ ]:
split_sets(test_number,"/content/w_test")

In [ ]:
bs=16
txt_proc = [
    TokenizeProcessor(tokenizer=Tokenizer(lang='es') ),
    NumericalizeProcessor(max_vocab=10000 )
]

#https://docs.fast.ai/text.data.html#TokenizeProcessor

In [ ]:
data_lm = (TextList.from_folder(path_art,processor=txt_proc)
          filter_by_folder(include=['train', 'test']) 
          .split_subsets(train_size=0.8, valid_size=0.2,seed=100)
          .label_for_lm()           
          #predicting the next word given the previous ones
          .databunch(bs=bs))

In [ ]:
data_lm.show_batch()

In [ ]:
data_lm.save('data_lm.pkl')

In [ ]:
data_lm = load_data(path_art, 'data_lm.pkl', bs=bs)